In [1]:
import numpy as np
import ot
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import accuracy_score
from scipy.io import savemat
import glob
import os
from sklearn.metrics import classification_report
import cv2,os
from sklearn.cluster import SpectralClustering
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns
import warnings
import scipy as sp
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
import sys
sys.path.append('../')
from HOTDA import *
warnings.filterwarnings("ignore")
%matplotlib notebook

In [4]:
print('training..')
Score_mean=[]
domain=[]
for file in glob.glob(os.path.join('data/', '', '*')):
    data    = sp.io.loadmat(file)
    Xsource = data['Xsource'].astype(float)
    Ysource = data['Ysource'].astype(float)
    XTarget = data['XTarget'].astype(float)
    Xtest   = data['Xtest'].astype(float)
    Ytest   = data['Ytest'].astype(float)
    Score=[]
    for i in range(10):
        clustering  = SpectralClustering(n_clusters=10, eigen_solver='arpack', assign_labels='kmeans',affinity="nearest_neighbors",n_neighbors=7).fit(XTarget)
        S,a,mu,y_Transported_S=Source_target_processing(Xsource,Ysource[0])
        T,b,nu,yxss=Source_target_processing(XTarget,clustering.labels_)
        HOT,W=Hot(S,a,mu,T,b,nu,100,100)
        X_Transported_S=Mapping(S,T,a,b,HOT,17)
        classifier =  KNeighborsClassifier(n_neighbors=1)
        classifier.fit(X_Transported_S,y_Transported_S)
        Score.append(classifier.score(Xtest,Ytest[0]))
    Score_mean.append(np.array(Score).mean())
    domain.append(''+file.split('_')[0][5]+'->'+file.split('_')[1][0])    

training..


In [5]:
df = pd.DataFrame({
     'Domain': domain,
     'Score': Score_mean})
df.to_csv("Result.csv", sep=';', index=False)

In [6]:
df

,Domain,Score
0,A->C,0.800000
1,A->D,0.925234
2,A->W,0.962449
3,C->A,0.921806
4,C->D,0.844860
5,C->W,0.945306
6,D->A,0.913326
7,D->C,0.776887
8,D->W,0.960816
9,W->A,0.918502
